In [1]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1', 'Game2', 'Game3', 'Game4', 'Game5']
Opp = 'Celtics'
Team = 'Heat'

In [2]:
players = ['adebayo', 'butler', 'highsmith', 'love', 'lowry', 'martin', 'robinson', 'strus', 'vincent', 'zeller', 'team', 'opp']

In [24]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                
                # For game 1, set the new team file as the overall and continue to the opponent
                if game[0:5] == 'Game1':
                    ovr_player_data['team'] = newplayer
                    newplayer.to_csv(f'!Data/Team.csv') # Write the new team file to the overall data folder
                    continue
                    
                # Load overall data for the team
                ovrplayerFile = (f'!Data/Team.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                    
                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall data frames can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer # Reassign the ovr_data_player['team'] data to the updated data
                    ovrplayer.to_csv('!Data/Team.csv') # Write the new team file to the overall data folder
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Team file is handled, so continue to the opponent file

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                    
                # For game 1, set the new opponent file as the overall and continue to the next player
                if game[0:5] == 'Game1':
                    ovr_player_data['opp'] = newplayer
                    newplayer.to_csv(f'!Data/Opp.csv') # Write the new opponent file into the overall data directory
                    continue
                    
                # Load overall data for the opponent
                ovrplayerFile = (f'!Data/Opp.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary

                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv('!Data/Opp.csv') # Write the updated opponent file to the overall data directory
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Opponent file is handled, continue and exit the loop
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
            
            # For game 1, set the new player file as the overall and continue to the next player
            if game == 'Game1':
                ovr_player_data[player] = newplayer
                newplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
                
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
            
            if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Highsmith file found for Game1.
No Highsmith file found for Game2.
No Highsmith file found for Game3.
No Highsmith file found for Game4.


In [4]:
newButlerStats = PPP(new_player_data['butler'])
newButlerStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.33,27.3,0,16.7,0.0,23.1,0.0,2,0.0,1,0.0,0.67,33.3,33.3,2,50.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.67,13.6,1,50.0,N/A,0.0,N/A,0,N/A,0,N/A,0.67,33.3,50.0,1,100.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.48,39.8,0,71.4,1.18,52.0,60.0,4,75.0,1,0.0,2.5,22.2,100.0,1,100.0,1,100.0
TRANSITION,2.33,15.6,0,100.0,2.05,18.8,100.0,2,100.0,0,N/A,3.0,11.1,100.0,0,N/A,1,100.0
Attacking Closeouts,0.0,4.5,1,N/A,0.0,7.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [5]:
butlerStats = PPP(ovr_player_data['butler'])
butlerStats.to_csv('!PPP Stats/butlerStats.csv')
butlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.96,27.0,6,47.1,1.06,22.5,45.0,19,47.4,1,0.0,0.82,36.0,50.0,8,62.5,6,33.3
PNR Screener,0.0,0.6,0,0.0,0.0,0.9,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,0.6,0,0.0,0.0,0.9,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.0,3.1,1,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,9.3,50.0,1,100.0,3,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.13,51.5,4,48.6,0.89,55.5,38.5,44,43.2,8,12.5,1.75,43.4,75.0,6,83.3,14,71.4
TRANSITION,1.64,7.5,2,100.0,1.58,5.8,100.0,4,100.0,0,N/A,1.7,10.9,100.0,2,100.0,2,100.0
Attacking Closeouts,0.67,1.8,1,50.0,0.67,2.8,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.67,1.8,0,100.0,2.67,2.8,100.0,1,100.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [6]:
newAdebayoStats = PPP(new_player_data['adebayo'])
newAdebayoStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,4.0,0,100.0,2.0,5.6,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.8,20.0,0,80.0,1.5,22.2,75.0,4,75.0,0,N/A,3.0,14.3,100.0,0,N/A,1,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.25,16.0,0,50.0,0.0,5.6,0.0,1,0.0,0,N/A,1.67,42.9,66.7,2,50.0,1,100.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.4,40.0,5,40.0,0.44,50.0,40.0,5,40.0,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
TRANSITION,2.0,12.0,0,100.0,2.0,11.1,100.0,2,100.0,0,N/A,2.0,14.3,100.0,1,100.0,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [7]:
adebayoStats = PPP(ovr_player_data['adebayo'])
adebayoStats.to_csv('!PPP Stats/adebayoStats.csv')
adebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.51,4.1,0,50.0,1.51,6.3,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.38,19.9,5,82.4,1.34,22.2,78.6,14,78.6,0,N/A,1.5,15.4,100.0,0,N/A,3,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.92,17.4,1,31.2,0.53,5.1,0.0,1,0.0,0,N/A,1.01,40.7,33.3,3,33.3,12,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.97,42.2,6,51.3,0.9,48.2,46.4,28,46.4,0,N/A,1.17,30.8,63.6,9,77.8,2,0.0
TRANSITION,1.8,8.8,1,88.9,1.33,8.1,80.0,5,80.0,0,N/A,2.5,10.3,100.0,2,100.0,2,100.0
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
newLoveStats = PPP(new_player_data['love'])
newLoveStats.to_csv(f'Love/PPP Stats/{prevGames[-1] + Opp}.csv')
newLoveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,0.0,42.9,0,0.0,0.0,42.9,0.0,3,0.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,1.0,28.6,0,50.0,1.0,28.6,50.0,1,100.0,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [9]:
loveStats = PPP(ovr_player_data['love'])
loveStats.to_csv('!PPP Stats/loveStats.csv')
loveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,6.7,1,0.0,0.0,4.3,0.0,1,0.0,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,3.3,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Screener,0.0,3.3,0,0.0,0.0,4.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.5,26.7,0,25.0,0.29,30.4,14.3,7,14.3,0,N/A,2.0,14.3,100.0,1,100.0,0,N/A
TRANSITION,1.6,16.7,0,60.0,1.5,17.4,50.0,0,N/A,4,50.0,2.0,14.3,100.0,1,100.0,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,23.3,1,83.3,1.83,26.1,80.0,1,100.0,4,75.0,3.0,14.3,100.0,0,N/A,1,100.0


In [10]:
newLowryStats = PPP(new_player_data['lowry'])
newLowryStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.8,62.5,3,100.0,1.0,40.0,100.0,1,100.0,0,N/A,0.67,100.0,100.0,1,100.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,12.5,1,N/A,0.0,20.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,12.5,0,0.0,0.0,20.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,12.5,0,0.0,0.0,20.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [11]:
lowryStats = PPP(ovr_player_data['lowry'])
lowryStats.to_csv('!PPP Stats/lowryStats.csv')
lowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.68,47.5,6,40.9,0.65,50.0,33.3,10,40.0,5,20.0,0.73,44.0,57.1,6,66.7,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,3.4,0,0.0,0.0,2.9,0.0,0,N/A,1,0.0,0.0,4.0,0.0,0,N/A,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.08,12.6,3,75.0,0.4,14.7,50.0,2,50.0,0,N/A,2.46,9.8,100.0,1,100.0,1,100.0
TRANSITION,1.42,20.3,1,63.6,1.0,8.8,33.3,2,0.0,1,100.0,1.56,36.0,75.0,5,80.0,3,66.7
Attacking Closeouts,0.67,5.1,1,50.0,1.0,5.9,50.0,2,50.0,0,N/A,0.0,4.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.2,8.5,0,40.0,1.2,14.7,40.0,0,N/A,5,40.0,N/A,0.0,N/A,0,N/A,0,N/A


In [12]:
newMartinStats = PPP(new_player_data['martin'])
newMartinStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,14.3,1,0.0,0.0,15.4,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,7.1,0,0.0,0.0,7.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.9,63.4,1,42.9,1.02,60.6,42.9,5,60.0,2,0.0,0.0,100.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,3.0,14.3,0,100.0,3.0,15.4,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [13]:
martinStats = PPP(ovr_player_data['martin'])
martinStats.to_csv('!PPP Stats/martinStats.csv')
martinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,5.7,1,66.7,1.0,6.8,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.67,4.3,0,33.3,0.67,5.1,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.0,4.3,0,66.7,2.0,5.1,66.7,1,0.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.5,2.9,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.5,18.2,50.0,0,N/A,2,50.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.67,21.3,2,33.3,0.84,20.1,36.4,7,57.1,4,0.0,0.0,27.3,0.0,0,N/A,1,0.0
TRANSITION,1.4,14.3,1,77.8,1.25,13.6,71.4,6,83.3,1,0.0,2.0,18.2,100.0,2,100.0,0,N/A
Attacking Closeouts,1.8,14.3,0,80.0,1.5,13.6,75.0,7,85.7,1,0.0,3.0,18.2,100.0,0,N/A,2,100.0
Catch & Shoot,1.9,28.6,0,65.0,1.84,32.2,63.2,1,100.0,18,61.1,3.0,9.1,100.0,0,N/A,1,100.0


In [14]:
newRobinsonStats = PPP(new_player_data['robinson'])
newRobinsonStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.33,20.0,0,100.0,2.0,11.1,100.0,1,100.0,0,N/A,2.5,33.3,100.0,1,100.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.67,40.0,1,80.0,1.75,44.4,75.0,3,66.7,1,100.0,1.5,33.3,100.0,0,N/A,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,13.3,0,0.0,0.0,22.2,0.0,1,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.0,13.3,0,100.0,2.0,11.1,100.0,1,100.0,0,N/A,2.0,16.7,100.0,1,100.0,0,N/A
Attacking Closeouts,2.5,13.3,0,100.0,3.0,11.1,100.0,0,N/A,1,100.0,2.0,16.7,100.0,1,100.0,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [15]:
robinsonStats = PPP(ovr_player_data['robinson'])
robinsonStats.to_csv('!PPP Stats/robinsonStats.csv')
robinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.88,19.5,0,75.0,1.25,13.8,50.0,1,100.0,3,33.3,2.5,33.3,100.0,2,100.0,2,100.0
PNR Screener,0.0,2.4,0,0.0,0.0,3.4,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.15,33.9,1,50.0,1.18,37.9,45.5,3,66.7,8,37.5,1.04,24.0,100.0,0,N/A,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.67,7.3,0,33.3,0.67,10.3,33.3,2,50.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.0,9.8,1,66.7,1.0,6.9,50.0,1,100.0,1,0.0,1.0,16.7,100.0,1,100.0,0,N/A
Attacking Closeouts,1.4,12.2,1,75.0,3.0,3.4,100.0,0,N/A,1,100.0,1.0,33.3,66.7,3,66.7,0,N/A
Catch & Shoot,3.0,4.9,0,100.0,3.0,6.9,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [16]:
newStrusStats = PPP(new_player_data['strus'])
newStrusStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.25,50.0,0,50.0,1.5,40.0,50.0,0,N/A,2,50.0,1.0,66.7,50.0,2,50.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,25.0,0,0.0,0.0,20.0,0.0,0,N/A,1,0.0,0.0,33.3,0.0,1,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,12.5,0,0.0,0.0,20.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
strusStats = PPP(ovr_player_data['strus'])
strusStats.to_csv('!PPP Stats/strusStats.csv')
strusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.43,30.4,1,61.5,1.6,26.3,66.7,2,100.0,7,57.1,1.0,50.0,50.0,3,66.7,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.25,17.1,0,0.0,0.0,10.5,0.0,0,N/A,4,0.0,0.52,48.5,0.0,2,0.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,4.3,0,0.0,0.0,5.3,0.0,1,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.07,14.7,0,100.0,2.07,17.8,100.0,4,100.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.0,8.7,1,66.7,1.0,10.5,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.65,17.1,0,57.1,1.71,18.4,57.1,0,N/A,7,57.1,1.14,11.0,N/A,0,N/A,0,N/A


In [18]:
newVincentStats = PPP(new_player_data['vincent'])
newVincentStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [19]:
vincentStats = PPP(ovr_player_data['vincent'])
vincentStats.to_csv('!PPP Stats/vincentStats.csv')
vincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.46,45.7,1,66.7,1.64,47.0,70.6,11,81.8,6,50.0,0.8,41.7,50.0,3,66.7,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.69,5.6,1,0.0,0.69,7.4,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.39,15.2,3,0.0,0.52,14.8,0.0,1,0.0,2,0.0,0.0,16.7,N/A,0,N/A,0,N/A
TRANSITION,1.5,7.8,0,50.0,3.0,2.6,100.0,0,N/A,1,100.0,1.0,25.0,33.3,0,N/A,3,33.3
Attacking Closeouts,1.6,7.4,0,50.0,1.45,7.1,0.0,0,N/A,1,0.0,2.0,8.3,100.0,1,100.0,0,N/A
Catch & Shoot,1.5,15.7,0,50.0,1.5,20.5,50.0,0,N/A,8,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [20]:
newZellerStats = PPP(new_player_data['zeller'])
newZellerStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,33.3,0,100.0,2.0,50.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [21]:
zellerStats = PPP(ovr_player_data['zeller'])
zellerStats.to_csv('!PPP Stats/zellerStats.csv')
zellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.8,33.3,1,50.0,1.0,20.0,50.0,2,50.0,0,N/A,0.67,60.0,50.0,1,100.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,6.7,0,100.0,2.0,10.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,5.9,0,N/A,0.0,8.8,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,6.7,0,0.0,0.0,10.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [22]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.95,18.9,4,52.9,0.82,13.6,44.4,6,50.0,3,33.3,1.1,33.3,62.5,6,66.7,2,50.0
PNR Screener,2.0,6.3,0,85.7,1.83,7.4,83.3,5,80.0,1,100.0,3.0,3.3,100.0,0,N/A,1,100.0
DHO Ball Handler,1.11,8.1,1,50.0,1.17,7.4,50.0,4,50.0,2,50.0,1.0,10.0,50.0,1,0.0,1,100.0
DHO Screener,1.0,6.3,1,50.0,0.0,1.2,0.0,1,0.0,0,N/A,1.17,20.0,60.0,3,66.7,2,50.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.76,32.1,7,42.3,0.7,39.1,37.5,20,45.0,4,0.0,1.25,13.3,100.0,1,100.0,1,100.0
TRANSITION,1.92,9.4,0,90.0,1.75,9.2,85.7,7,85.7,0,N/A,2.33,10.0,100.0,2,100.0,1,100.0
Attacking Closeouts,1.25,3.6,1,66.7,1.0,3.7,50.0,1,0.0,1,100.0,2.0,3.3,100.0,1,100.0,0,N/A
Catch & Shoot,2.0,6.3,0,71.4,2.0,8.6,71.4,1,100.0,6,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [25]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.26,14.1,2,60.0,0.95,11.2,50.0,8,50.0,0,N/A,1.62,20.0,71.4,3,66.7,4,75.0
PNR Screener,2.0,4.8,0,83.3,2.25,4.8,100.0,3,100.0,1,100.0,1.5,5.0,50.0,0,N/A,2,50.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.42,20.4,2,63.6,1.3,20.1,60.0,11,63.6,4,50.0,1.66,21.1,71.4,1,100.0,6,66.7
TRANSITION,1.02,12.7,2,50.0,1.02,8.2,50.0,6,50.0,0,N/A,1.01,22.2,50.0,3,66.7,3,33.3
Attacking Closeouts,1.12,16.5,0,41.2,1.33,16.1,50.0,7,42.9,3,66.7,0.71,17.5,28.6,2,50.0,5,20.0
Catch & Shoot,1.16,23.6,1,40.7,1.2,33.7,40.7,1,100.0,26,38.5,0.0,2.5,N/A,0,N/A,0,N/A


In [23]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.12,20.9,15,54.4,1.2,19.4,52.9,48,58.3,22,40.9,0.97,24.7,57.6,22,68.2,11,36.4
PNR Screener,1.13,5.9,7,64.3,1.14,6.0,60.9,19,68.4,4,25.0,1.1,5.7,80.0,1,100.0,4,75.0
DHO Ball Handler,0.82,5.3,3,30.8,0.92,5.4,33.3,5,40.0,16,31.2,0.57,5.0,20.0,2,0.0,3,33.3
DHO Screener,0.94,4.6,2,34.8,0.42,1.1,0.0,1,0.0,1,0.0,1.05,13.0,38.1,4,50.0,17,35.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.97,29.3,18,46.2,0.82,31.2,38.2,94,43.6,16,6.2,1.44,24.9,71.4,17,82.4,18,61.1
TRANSITION,1.56,10.9,6,78.2,1.49,9.1,75.0,23,82.6,9,55.6,1.67,15.3,82.6,13,92.3,10,70.0
Attacking Closeouts,1.31,4.9,4,66.7,1.24,5.1,61.1,15,66.7,3,33.3,1.5,4.5,83.3,4,75.0,2,100.0
Catch & Shoot,1.84,9.3,1,64.8,1.81,12.4,63.5,3,100.0,49,61.2,2.43,1.6,100.0,0,N/A,2,100.0


In [26]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.02,14.2,11,48.7,0.84,9.6,41.2,24,50.0,10,20.0,1.18,23.8,54.8,19,73.7,23,39.1
PNR Screener,1.49,5.4,5,71.4,1.6,5.7,72.7,13,84.6,9,55.6,1.2,4.8,66.7,0,N/A,6,66.7
DHO Ball Handler,2.33,0.9,0,83.3,2.25,0.9,75.0,0,N/A,4,75.0,2.5,1.0,100.0,1,100.0,1,100.0
DHO Screener,1.25,0.6,0,50.0,0.0,0.2,0.0,0,N/A,1,0.0,1.67,1.4,66.7,1,100.0,2,50.0
DBL Ball Handler,1.2,0.8,0,60.0,0.67,0.7,33.3,3,33.3,0,N/A,2.0,1.0,100.0,2,100.0,0,N/A
DBL Screener,2.0,0.3,0,100.0,2.0,0.4,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,19.4,13,47.6,1.02,17.4,50.0,50,50.0,12,50.0,0.98,23.7,43.9,17,64.7,24,29.2
TRANSITION,1.1,16.8,11,50.6,1.04,14.2,48.0,37,54.1,13,30.8,1.18,22.2,54.3,17,82.4,18,27.8
Attacking Closeouts,1.05,14.2,6,44.7,0.95,14.8,42.3,38,52.6,14,14.3,1.29,13.0,50.0,5,80.0,19,42.1
Catch & Shoot,0.94,17.6,3,32.4,0.95,24.9,32.4,1,100.0,107,31.8,0.5,1.9,33.3,1,100.0,2,0.0
